In [1]:
import os
import sys

notebook_dir = os.path.abspath('')
sys.path.append(os.path.join(notebook_dir, '..'))

import numpy as np

from sklearn.metrics import accuracy_score
from keras_core import utils
from aepy.data.datasets import load_MNIST
from aepy.data.utils import evaluate
from aepy.models.base.default_architectures import (Decoder_Conv_MNIST, Decoder_MLP,
                                               Encoder_Conv_MNIST, Encoder_MLP)
from aepy.models.vae.vae_model import VAE
from aepy.pipelines.training import TrainingPipeline

Using TensorFlow backend


2023-11-21 11:03:11.724338: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# Load MNIST dataset
x_train, y_train, x_test, y_test = load_MNIST(persistant=True)

# Obtaint number of clasess
n_classes = len(set(y_train))

# Convert labels to categorical
y_train = utils.to_categorical(y_train, n_classes)
y_test = utils.to_categorical(y_test, n_classes)

2023-11-21 11:03:34 [INFO]: train-images-idx3-ubyte.gz already exists.
2023-11-21 11:03:34 [INFO]: train-labels-idx1-ubyte.gz already exists.
2023-11-21 11:03:34 [INFO]: t10k-images-idx3-ubyte.gz already exists.
2023-11-21 11:03:34 [INFO]: t10k-labels-idx1-ubyte.gz already exists.


In [3]:
train_data = dict(data=x_train.astype(float), labels=y_train)
test_data = dict(data=x_test.astype(float), labels=y_test)

In [4]:
# Model creation
model = VAE(input_dim=(x_train.shape[0], x_train.shape[1]), latent_dim=10, downstream_task='classification',
            encoder=Encoder_Conv_MNIST, decoder=Decoder_Conv_MNIST, layers_conf=[32, 64], n_classes=n_classes)

2023-11-21 11:03:38 [INFO]: Classificator available for the latent space of the autoencoder


In [5]:
pipe = TrainingPipeline(name='training_pipeline',
                        model=model, num_epochs=2)

trained_model = pipe(train_data=train_data)

2023-11-21 11:03:41 [INFO]: +++ training_pipeline +++
2023-11-21 11:03:41 [INFO]: Creating folder in ../output_dir/training_pipeline_2023-11-21_11-03-41


Epoch 1/2


2023-11-21 11:04:20.700637: I external/local_xla/xla/service/service.cc:168] XLA service 0x7f78ec015a20 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2023-11-21 11:04:20.700715: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2023-11-21 11:04:20.923221: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1700561063.999875   13952 device_compiler.h:187] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2023-11-21 11:04:24.004595: E external/local_xla/xla/stream_executor/stream_executor_internal.h:181] SetPriority unimplemented for this stream.



Epoch 1: total_loss improved from inf to 11131115536384.00000, saving model to ../output_dir/training_pipeline_2023-11-21_11-03-41/model.weights.h5
469/469 - 123s - 262ms/step - clf_loss: 1556.6891 - kl_loss: 11131115536384.0000 - reconstruction_loss: 7217.3125 - total_loss: 11131115536384.0000
Epoch 2/2

Epoch 2: total_loss improved from 11131115536384.00000 to 142058.39062, saving model to ../output_dir/training_pipeline_2023-11-21_11-03-41/model.weights.h5
469/469 - 78s - 166ms/step - clf_loss: 2.7070 - kl_loss: 134838.6719 - reconstruction_loss: 7216.9019 - total_loss: 142058.3906


In [6]:
y_hat = trained_model.predict(test_data)

evaluate(y_true=np.argmax(test_data['labels'], axis=1), 
         y_hat=np.argmax(y_hat['clf'], axis=1),
         sel_metric=accuracy_score)

313/313 ━━━━━━━━━━━━━━━━━━━━ 11s 27ms/step
test set results: [
	 accuracy_score: 0.2278 
]


0.2278